# Old stuff

In [6]:
import spacy
spacy.load('en_core_web_md')
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator
from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
from epitator.count_annotator import CountAnnotator
from epitator.date_annotator import DateAnnotator
from boilerpipe.extract import Extractor
from itertools import groupby
import datetime
import sysgit s
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np
import epitator

## Read in Ereignisdatenbank

In [57]:
ereignisdatenbank = pd.read_csv("Ereignisse_utf8.csv",sep=";")

In [7]:
ereignisdatenbank.columns = list(map(lambda x:x.strip(" "),ereignisdatenbank.columns))

In [8]:
countries_not_null = ereignisdatenbank[pd.notnull(ereignisdatenbank["Ausgangs- bzw. Ausbruchsland"])]

In [9]:
countries = countries_not_null["Ausgangs- bzw. Ausbruchsland"].copy(deep=True)
countries = list(map(lambda x:x.strip(" "),countries))

# Let the testing begin


## Ontology/Comparison (Transformed to .py until here)

In [531]:
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import location_contains
doc = AnnoDoc("I live in Munic!")
doc.add_tiers(GeonameAnnotator())
annotations = doc.tiers["geonames"]
geoname = annotations[0]
geoname.to_dict()

{'label': 'Munich',
 'textOffsets': [[10, 15]],
 'geoname': {'geonameid': '2867714',
  'name': 'Munich',
  'feature_code': 'PPLA',
  'country_code': 'DE',
  'admin1_code': '02',
  'admin2_code': '091',
  'admin3_code': '09162',
  'admin4_code': '09162000',
  'longitude': 11.57549,
  'latitude': 48.13743,
  'population': 1260391,
  'asciiname': 'Munich',
  'names_used': 'Munic',
  'name_count': 88,
  'country_name': 'Federal Republic of Germany',
  'admin1_name': 'Bavaria',
  'admin2_name': 'Upper Bavaria',
  'admin3_name': 'Kreisfreie Stadt München',
  'parents': [],
  'score': 0.21453999698331166}}

In [ ]:
# # FOR TESTING. RETURNS TUPLE WITH ABBREVIATION AND TRANSLATION
# # Takes a list of not matched/translated entries and tries to match them to the wikipedia table and find the full name
# countries_not_translated = [entry for entry in countries_unique \
#                             if entry not in wikipedia_country_list["state_name_de"].tolist()]
# def translate_abbreviation(to_translate):
#     abb_to_country = []
#     if type(to_translate) == str:
#         to_translate = [to_translate]
#     for column in ["wiki_abbreviations","inoff_abbreviations"]:
#         for potential_abbreviation in to_translate:
#             if type(potential_abbreviation) == str:
#                 for i, abbreviation in enumerate(wikipedia_country_list[column]):
#                     if potential_abbreviation in abbreviation:
#                         abb_to_country.append((potential_abbreviation,\
#                                                wikipedia_country_list["translation_state_name"].tolist()[i]))
#                         to_translate.remove(potential_abbreviation)
#             elif type(potential_abbreviation) == list:
#                 abb_to_country.append(translate_abbreviation(potential_abbreviation))
#     return(abb_to_country,to_translate)

# abbreviation_tuple, countries_not_translated = translate_abbreviation(countries_not_translated)

# #abbreviation_tuple
# #print("****************************************************************************************************************")
# #print(countries_not_translated)

In [536]:
#TODO if abbreviation found, don't continue searching
def translate_abbreviation(to_translate):
    """Takes a list of countries and/or abbreviations and translates the abbreviations to the full state name"""
    to_return = []
    if type(to_translate) == str:
        to_translate = [to_translate]
    for potential_abbreviation in to_translate:
        if type(potential_abbreviation) == str and not re.findall(r"([^A-Z]+)",potential_abbreviation):
            
            # First check the official abrev. than the self created ones e.g. VAE for the Emirates
            for column in ["wiki_abbreviations","inoff_abbreviations"]:
                for i, abbreviation in enumerate(wikipedia_country_list[column]):
                    if potential_abbreviation in abbreviation:
                        to_return.append(wikipedia_country_list["state_name_de"].tolist()[i])
        elif type(potential_abbreviation) == list:
            list_entry = [translate_abbreviation(nested_entry) for nested_entry in potential_abbreviation]
            flattened = [entry for sublist in list_entry for entry in sublist]
            to_return.append(flattened)
        else:
            to_return.append(potential_abbreviation)
    return to_return

In [537]:
# Test for translate_abbreviation()
example_to_abbreviate = ["USA","VAE",'Italien', "DR Cong",["Deutschland", "EU"],["Belgien","DRC"]]
desired_output = ['Vereinigte Staaten','Vereinigte Arabische Emirate','Italien','DR Cong',
                  ['Deutschland', 'Europäische Union'],
                  ['Belgien', 'Kongo, Demokratische Republik']]
if deep_eq(translate_abbreviation(example_to_abbreviate),desired_output):
    print("Test succesful")
else:
    Print("Test failed")

Test succesful


In [ ]:
## SIMPLE TRANSLATION. FAST BUT DOES NOT TRANSLATE LISTS OF LISTS
# # Translate German entries of Ereignisdatenbank to English. Might be inefficient since I go through the wiki list
# # entirely which is longer then the list of countries to translate
# translated_ereignisdatenbank_countries = [(entry,wikipedia_country_list["translation_state_name"].tolist()[indx])\
#                                           for indx,entry \
#                                           in enumerate(wikipedia_country_list["state_name_de"].tolist())\
#                                           if entry in countries_unique]

In [538]:
from didyoumean import didyoumean

def translate(countries_unique):
    """Translate German entries of Ereignisdatenbank to English an returns tuple 
    of German word and English translation(s if ambigious)""" 

    continents = ["europa","africa","america","australien","asia"]
    translated_ereignisdatenbank_countries = []
    state_name_de = wikipedia_country_list["state_name_de"].tolist()
    full_state_name_de = wikipedia_country_list["full_state_name_de"].tolist()
    translation = wikipedia_country_list["translation_state_name"].tolist()
    
    if type(countries_unique) == str:
        countries_unique = [countries_unique]
    
    for entry in countries_unique:
        if type(entry) == str:
            sucessfull_search = list(filter(lambda x: re.findall(entry,x),state_name_de))
            if sucessfull_search:
                found = [translation[state_name_de.index(entry)] for entry in sucessfull_search]
                if len(found) == 1:
                    translated_ereignisdatenbank_countries.append((entry,found[0]))
                else:
                    
                    # Check for idendity in not ambigious case otherwise e.g Niger --> (Niger, Nigeria)
                    identical = [found_ent for found_ent in found if entry == found_ent]
                    if identical:
                        translated_ereignisdatenbank_countries.append((entry,identical[0]))
                    else:
                        translated_ereignisdatenbank_countries.append((entry,found))
            else:
                
                # If entry not in state_name_de, search in full_state_name_de
                sucessfull_search = list(filter(lambda x: re.findall(entry,x),full_state_name_de))
                if sucessfull_search:
                    found = [translation[full_state_name_de.index(entry)] for entry in sucessfull_search]
                    if len(found) == 1:
                        translated_ereignisdatenbank_countries.append((entry,found[0]))
                    else:
                        translated_ereignisdatenbank_countries.append((entry,found))
                else:
                    sucessfull_search_en = list(filter(lambda x: re.findall(entry,x),translation))
                    if sucessfull_search_en:
                        found = [state_name_de[translation.index(entry)] for entry in sucessfull_search_en]
                        if len(found) == 1:
                            translated_ereignisdatenbank_countries.append((found[0],entry))
                        else:
                            translated_ereignisdatenbank_countries.append((found,entry))
                    else:
                        
                        # If there was not match at all, check for spelling mistakes
                        did_u_mean = didyoumean.didYouMean(entry,state_name_de)
                        
                        """Exlude words with continent names since there are countries with a continent name
                        but there are also entries in the Ereignisdatenbank that mean the whole country. They
                        are not matched (e.g. Nordafrika) and there is must not be a match, otherwise 
                        didYouMean would falsly return Südafrika.
                        """
                        if did_u_mean and (did_u_mean not in continents):
                            translated_ereignisdatenbank_countries.append(translate(did_u_mean))
                        else:
                            translated_ereignisdatenbank_countries.append(entry)
                                                            
        elif type(entry) == list:
            translated_ereignisdatenbank_countries.append(translate(entry))
        else:
            translated_ereignisdatenbank_countries.append(entry)
    return translated_ereignisdatenbank_countries

In [23]:
a = "hello"
a = list(a)
a

['h', 'e', 'l', 'l', 'o']

In [65]:
ereignisdatenbank["Ausgangs- bzw. Ausbruchsland "].tolist()

['Nigeria',
 'Benin',
 'Liberia',
 'Ghana',
 'Fiji',
 'Zentralafrikanische Republik',
 'Angola',
 ' Äthiopien (AWD)',
 'Zambia',
 'Südafrika',
 'Simbabwe',
 'Tschad',
 'Namibia',
 'Yemen',
 'Sudan',
 'Indien',
 'Burundi',
 'Demokratische Republik Kongo',
 'Kenia',
 'Malawi',
 'Mosambik',
 'Nigeria',
 'Somalia',
 'Tanzania',
 'Uganda',
 'Oman ',
 'Saudi-Arabien ',
 'Deutschland',
 'Syrien',
 'Deutschland',
 'Südsudan',
 'Südafrika',
 'Südafrika',
 'Kenia',
 'Namibia',
 'Bangladesch',
 'Bangladesch',
 'Bangladesch',
 'Brasilien',
 'Brasilien',
 'Brasilien',
 'Nigeria',
 'Nigeria',
 'Nigeria',
 'La Reunion',
 'La Reunion',
 'La Reunion',
 'Liberia',
 'Demokratische Republik Kongo',
 'Zentralafrikanische Republik',
 'Namibia',
 'Uganda',
 'Australien',
 'Brasilien',
 'Pakistan',
 'La Reunion',
 'Madagaskar',
 'Südafrika',
 'Liberia',
 'Bolivien',
 'Simbabwe',
 'Uganda',
 'Angola',
 'Angola',
 'Demokratische Republik Kongo',
 'Malawi',
 'Mosambik',
 'Nigeria',
 'Nigeria',
 'Nigeria',
 'Nige

In [540]:
example_to_translate = ["Deutschland","Delaware",["Kongo","China"],"Niger"]
expected_result_translate = [('Deutschland', 'Germany'),
                             'Delaware',
                             [('Kongo',
                               ['Congo, Democratic Republic of the (Kinshasa)','Congo, Republic of (Brazzaville)']),
                              ('China','China')],
                            ('Niger', 'Niger')]
if deep_eq(translate(example_to_translate),expected_result_translate):
    print("Test succesfull")
else:
    print("Test failed")

Test succesfull


In [541]:
translated = translate(translate_abbreviation(clean_entries(countries)))
translated

[('Algerien', 'Algeria'),
 ('Peru', 'Peru'),
 ('Äthiopien', 'Ethiopia'),
 [('Ungarn', 'Hungary'),
  ('Serbien', 'Serbia'),
  ('Rumänien', 'Romania'),
  ('Italien', 'Italy'),
  ('Griechenland', 'Greece')],
 ('Sri Lanka', 'Sri Lanka'),
 ('El Salvador', 'El Salvador'),
 ('Jemen', 'Yemen'),
 ('Namibia', 'Namibia'),
 ('Simbabwe', 'Zimbabwe'),
 ('Kongo, Demokratische Republik',
  'Congo, Democratic Republic of the (Kinshasa)'),
 ('Angola', 'Angola'),
 ('Burundi', 'Burundi'),
 ('Vereinigte Staaten', 'United States'),
 ('Kolumbien', 'Colombia'),
 ('Bolivien', 'Bolivia'),
 ('Schweiz', 'Switzerland'),
 ('Südsudan', 'South Sudan'),
 ('Haiti', 'Haiti'),
 ('Venezuela', 'Venezuela'),
 ('Niger', 'Niger'),
 ('Ghana', 'Ghana'),
 ('Costa Rica', 'Costa Rica'),
 ('China', 'China'),
 ('Bangladesch', 'Bangladesh'),
 ('Somalia', 'Somalia'),
 ('Syrien', 'Syria'),
 'Französich Polynesien',
 ('Kanada', 'Canada'),
 ('Tansania', 'Tanzania'),
 ('Nigeria', 'Nigeria'),
 ('Taiwan', 'Taiwan oder Republic of China'),
 

In [542]:
flattened = [entry if type(sublist) == list else sublist for sublist in translated for entry in sublist ]
countries_not_translated = set([entry for entry in flattened if type(entry) == str])
countries_not_translated

{'DR Congo',
 'DRCongo',
 'Delaware',
 'Französich Polynesien',
 'Französiche Guyana',
 'Französisch-Polynesien',
 'La Reunion',
 'Nordeuropa',
 'Typhus',
 'VAE Dubai'}